In [ ]:
import tqdm
import os
import pickle
import re
import numpy as np

In [ ]:
import glob, re

sentences = []
for f in glob.glob("PMC000xxxxxx/*.txt"):
    with open(f, "r", encoding="latin-1") as file:
        content = file.read()
    for sentence in content.split("\n"):
        if sentence.strip() and not sentence.startswith("==== "):
            sentence = sentence.strip()

            sentences+=re.split(r"[{. }!?;]\s+", sentence.replace("\t", " "))
sentences = [s.lower() for s in sentences]

In [ ]:
term_lists = [
[[["bulla", "bullae"]], [["blister", "blisters"],]],
[[["candida albicans"], ["candidiasis"]], [["thrush",]]],
[[["carbohydrate", "carbohydrates"]], [["carb", "carbs"]]],
[[["chemotherapy", "chemotherapies"]], [["chemo", "chemoes"]]],
[[["chronic pain", "chronic pains"]], [["persistent pain", "persistent pains"]]],
[[["comedo", "comedos"]], [["whitehead", "whiteheads"]]],
[[["dermis",], ["epidermis",]], [["skin"]]],
[[["dyspepsia"]], [["indigestion"]]],
[[["erythrocyte", "erythrocytes"]], [["red blood cell", "red blood cells"]]],
[[["febrile"]], [["feverish"]]],
[[["haemorrhage"]], [["heavy bleeding"]]],
[[["herpes zoster"]], [["chickenpox"]]],
[[["hypertension"]], [["high blood pressure", "high blood pressures"]]],
[[["hypotension"]], [["low blood pressure", "low blood pressures"]]],
[[["influenza"]], [["flu"]]],
[[["inhaler"]], [["puffer"]]],
[[["intestine"]], [["guts"]]],
[[["lethargy"]], [["tiredness"]]],
[[["leukocyte", "leukocytes"]], [["white blood cell", "white blood cells"]]],
[[["myocardial infarction"]], [["heart attack"]]],
[[["pneumonia"]], [["lung infection"]]],
[[["renal failure"]], [["kidney failure"]]],
[[["thrombocytopenia"]], [["low platelet count", "low platelet counts"]]],
[[["liposuction"]], [["lipo"]]],
[[["melanoma"]], [["skin cancer"]]],
]

n = 5
for bio_terms, usual_terms in term_lists:

    terms = bio_terms+usual_terms

    save_path = os.path.join("ngram_surprisals", "|".join([term[0] for term in terms]))


    if os.path.isfile(save_path): continue
    term_types = ["bio"]*len(bio_terms)+["usual"]*len(usual_terms)

    term_surprisals = {term[0]: [] for term in terms}

    pbar = tqdm.tqdm(terms, desc="|".join([term[0] for term in terms]))
    for term in pbar:
        if type(term)!=list: raise ValueError("Term must be a list")
        regex_str = fr'\b(?:{"|".join(term)})\b'
        
        idx = 0

        context_sentences = []
        term_list = []

        for sentence1, sentence2 in zip(sentences[:-1], sentences[1:]):

            matches = list(re.finditer(regex_str, sentence2))
            if matches:
                for match in matches:

                    context = " ".join((sentence1+" "+sentence2[:match.start()].strip()).split(" ")[-n:])

                    context_sentences.append(context)

        for idx, context in enumerate(context_sentences):
            regex_str = context.replace("(", "\(").replace(")", "\)").replace("?", "\?").replace("!", "\!").replace(",", "\,").replace(".", "\.").replace("[", "\[").replace("]", "\]").replace("{", "\{").replace("}", "\}").replace("*", "\*").replace("+", "\+").replace("$", "\$").replace("^", "\^").replace("|", "\|")
            all_sentence = " ".join(sentences)
            matches = [m for m in re.finditer(regex_str, all_sentence)]
            context_count = len(list(matches))
            if context_count==0: raise ValueError

            context_prob = context_count/len(sentences)
            
            term_context_count = 0
            regex_str = fr'\b(?:{"|".join(term)})\b'
            for match in matches:

                term_context_count+=len(list(re.finditer(regex_str, all_sentence[match.end():match.end()+max([len(t) for t in term])+1])))

            if term_context_count==0: raise ValueError
            term_prob = term_context_count/len(sentences)

            term_surprisal = -np.log(term_prob/context_prob)
            term_surprisals[term[0]].append(term_surprisal)

            if term_context_count>context_count:
                for context_sentence in context_sentences:
                    if len(list(re.finditer(regex_str, context_sentence)))!=0:
                        print(context_sentence)

                print(term[0], context_count, term_context_count)
                raise ValueError
            pbar.set_postfix({"progress": f"{idx+1}/{len(context_sentences)}"})

    with open(save_path, "wb") as f:

        pickle.dump((term_surprisals, term_types), f)
